# 分类模型:交通标识

In [1]:
# 把 maix_train 项目的classifier抽离出来
# 基准测试通过
# 4 分类：左右，前，有行人

## python包

In [2]:
import os, sys
import shutil
import time
import traceback

from train.classifier import Classifier
from train.utils.logger import Logger, Fake_Logger
from train.utils import gpu_utils, isascii

In [3]:
from train.classifier.instance import config

config.classifier_train_epochs = 40 # 40 修改训练轮数
config.classifier_train_batch_size = 5 # 5

## config setting

In [4]:
cur_time = time.strftime("%Y-%m-%d_%H_%M", time.localtime())
print(cur_time)

2021-09-23_12_05


In [5]:
ubuntu_path = f"out/imgn_jt_{cur_time}" #用来mac中生成kmodel
temp_saves_dir = f"../ubuntu/{ubuntu_path}/" #保存文件目录
datasets_dir = "/Users/liampro/Downloads/dataLake/images/图像分类/交通标识/clean4"#数据源
print(temp_saves_dir)

../ubuntu/out/imgn_jt_2021-09-23_12_05/


## 训练函数

In [6]:
def clear_save_path(temp_datasets_dir=""):
    if temp_datasets_dir =="":
        return False
    #clear
    if os.path.exists(temp_datasets_dir):
        shutil.rmtree(temp_datasets_dir)
    os.makedirs(temp_datasets_dir)
    return True

def creat_sample_images(obj,temp_datasets_dir="",sample_image_num = 20):
    dataset_sample_images_path = os.path.join(temp_datasets_dir, "sample_images")
    os.makedirs(dataset_sample_images_path)
    obj.get_sample_images(sample_image_num, dataset_sample_images_path)
    return True

In [7]:
def classifier_train(datasets_zip_path="",datasets_dir="",temp_datasets_dir=""):
    #config.classifier_train_epochs = 40 # 修改训练轮数
    
    #clear and creat
    clear_save_path(temp_datasets_dir)
    log = Logger(file_path=f"{temp_datasets_dir}train_log.log")
    
    def __on_train_progress(percent, msg):  # flag: progress
        percent = percent*0.97 + 1
        log.i(f"progress: {percent}%, {msg}")
    
    # 检测 GPU 可用,选择一个可用的 GPU 使用
    try:
        gpu = gpu_utils.select_gpu(memory_require = config.classifier_train_gpu_mem_require, tf_gpu_mem_growth=False)
    except Exception:
        gpu = None
    if gpu is None:
        if not config.allow_cpu:
            log.e("no free GPU")
            raise Exception((TrainFailReason.ERROR_NODE_BUSY, "node no enough GPU or GPU memory and not support CPU train"))
        log.i("no GPU, will use [CPU]")
    else:
        log.i("select", gpu)

    # set value
    best_h5_model_path  = os.path.join(temp_datasets_dir, "m_best.h5")
    final_h5_model_path = os.path.join(temp_datasets_dir, "m.h5")
    result_dir = os.path.join(temp_datasets_dir, "result")
    os.makedirs(result_dir)
    result_report_img_path = os.path.join(result_dir, "report.jpg")
    tflite_path = os.path.join(temp_datasets_dir, "m.tflite")
    
    # 启动训练
    try:
        classifier = Classifier(datasets_zip=datasets_zip_path, datasets_dir=datasets_dir, unpack_dir =temp_datasets_dir,
                                logger=log,
                                max_classes_num=config.classifier_train_max_classes_num,
                                min_images_num=config.classifier_train_one_class_min_img_num,
                                max_images_num=config.classifier_train_one_class_max_img_num,
                                allow_reshape=False)
    except Exception as e:
        log.e("train datasets not valid: {}".format(e))
        raise Exception(("TrainFailReason.ERROR_PARAM", "datasets not valid: {}".format(str(e))))
    try:
        #_base = "mobilenet_7_5_224_tf_no_top.h5"
        #classifier.train(epochs=config.classifier_train_epochs,can_gen=True, batch_size=config.classifier_train_batch_size, progress_cb= __on_train_progress,weights=_base)
        classifier.train(epochs=config.classifier_train_epochs,can_gen=False, batch_size=config.classifier_train_batch_size, progress_cb= __on_train_progress)
    except Exception as e:
        log.e("train error: {}".format(e))
        traceback.print_exc()
        raise Exception(("TrainFailReason.ERROR_INTERNAL", "error occurred when train, error: {}".format(str(e)) ))

    ## sample_image
    creat_sample_images(classifier,temp_datasets_dir,config.sample_image_num)
    
    ## creat tflite
    classifier.save(tflite_path=tflite_path)
    
    # 训练结束, 生成报告
    log.i("train ok, now generate report")
    classifier.report(result_report_img_path)
    
    return True, classifier, final_h5_model_path , classifier.get_labels()


In [8]:
## 清理隐藏文件
#cmd1 = f"rm -f {datasets_dir}/.DS_Store"
#os.popen(cmd1)
cmd = f"rm -f {datasets_dir}/*/.DS_Store"
f = os.popen(cmd)
print(f)

## 训练

In [9]:
# 配置文件：train.classifier.instance
result,_,model_path, labels = classifier_train(datasets_dir=datasets_dir,temp_datasets_dir=temp_saves_dir)

select gpu fail:NVML Shared Library Not Found


2021-09-23 12:05:14,795 - [INFO]:  no GPU, will use [CPU]
2021-09-23 12:05:18,718 - [INFO]:  train, labels:['3-man', '2-right', '1-forward', '0-left']
2021-09-23 12:05:18,719 - [DEBUG]:  train, datasets dir:/Users/liampro/Downloads/dataLake/images/图像分类/交通标识/clean4
2021-09-23 12:05:18.796803: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/k210v37/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 24)      648       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 24)      96        
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 24)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 24)      216       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 24)      96    

2021-09-23 12:05:19,436 - [INFO]:  train data:1388, valid data:152
/opt/anaconda3/envs/k210v37/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-23 12:05:19,541 - [INFO]:  train start
2021-09-23 12:05:19,541 - [INFO]:  progress: 1.0%, train start
2021-09-23 12:05:19.551817: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-23 12:05:19,577 - [INFO]:  epoch 0 start


Epoch 1/40
277/277 [==============================] - 25s 87ms/step - loss: 1.2337 - accuracy: 0.4208 - val_loss: 1.0998 - val_accuracy: 0.4667


...

2021-09-23 12:22:12,717 - [INFO]:  epoch 39 end: {'loss': 0.18881095945835114, 'accuracy': 0.9595082998275757, 'val_loss': 0.2502019703388214, 'val_accuracy': 0.9333333373069763}
2021-09-23 12:22:12,718 - [INFO]:  progress: 98.0%, train epoch end
2021-09-23 12:22:12,719 - [INFO]:  train end
2021-09-23 12:22:12,720 - [INFO]:  progress: 98.0%, train end
2021-09-23 12:22:12,788 - [INFO]:  save model as .tflite file
2021-09-23 12:22:16.145770: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/_l/dffcyljj723fg4wbqfxfqv640000gn/T/tmp_j84w54s/assets


2021-09-23 12:22:21.314799: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-09-23 12:22:21.314912: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-23 12:22:21.319705: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-09-23 12:22:22.323578: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-09-23 12:22:22.323595: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-09-23 12:22:22.340673: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `

Found 1540 images belonging to 4 classes.
308/308 [==============================] - 22s 71ms/step


2021-09-23 12:22:45,277 - [INFO]:  generate report image end
INFO:logger: generate report image end


In [10]:
print(model_path)
print(labels)

../ubuntu/out/imgn_jt_2021-09-23_12_05/m.h5
dict_keys(['0-left', '1-forward', '2-right', '3-man'])


## 生成kmodel

In [11]:
#直接生成kmodel:Mac开发环境要弄个docker跑linux命令，
# win,linux可以直接跑官方命令
# tool:https://github.com/kendryte/nncase/releases/tag/v0.1.0-rc5

exe_path = os.path.join(os.path.abspath('..'), "ubuntu")

docker_cmd=f"docker run -it -v {exe_path}:/train ubuntu /train/tokmodel.sh {ubuntu_path}"
print(docker_cmd)

# 在shell 运行 docker命令

docker run -it -v /Users/liampro/Downloads/pro/git/ai-test/k210/ubuntu:/train ubuntu /train/tokmodel.sh out/imgn_jt_2021-09-23_12_05


In [12]:
# 或者运行下面的shell
cmd_sh = f"{os.path.abspath('.')}/tools/kmodel/ncc-mac.sh {exe_path} {ubuntu_path}"
print(cmd_sh)
# f=os.popen(cmd)
# print(f.read())

/Users/liampro/Downloads/pro/git/ai-test/k210/k210_train/tools/kmodel/ncc-mac.sh /Users/liampro/Downloads/pro/git/ai-test/k210/ubuntu out/imgn_jt_2021-09-23_12_05


## 生成k210程序boot.py

In [13]:
#temp_saves_dir = "../ubuntu/out/yolo_1"
target_file = f"{temp_saves_dir}result/boot.py"
copy_file = f"cp -f data/template/imgn_boot.py {target_file}"
print(copy_file)

cp -f data/template/imgn_boot.py ../ubuntu/out/imgn_jt_2021-09-23_12_05/result/boot.py


In [14]:
f=os.popen(copy_file)
print(f.read()) 

In [15]:
# 更新 labels 
result_boot_py_path = target_file

img_size = [224,224]
#labels = ["right", "left", "forward"]
replace = 'labels = ["{}"]'.format('", "'.join(labels))

with open(result_boot_py_path) as f:
            boot_py = f.read()
with open(result_boot_py_path, "w") as f:
            target = 'labels = ["0", "1"] # labels'
            boot_py = boot_py.replace(target, replace)
            target = 'img_size = [224,224] # img_size'
            replace = 'img_size = [{},{}]'.format(img_size[0], img_size[1])
            boot_py = boot_py.replace(target, replace)
            f.write(boot_py)

## 最后生成的文件tree

## 到k210硬件运行